## RPFS Problem (TWCT objective) - Data treatment of result files 

In [95]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import os
from pathlib import Path

%matplotlib inline

In [96]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

### List files in the output folder 

In [97]:
rootfolder = os.path.join('..', '..', 'pfsp_experiments', 'run_ccg_rpfs_wct_global')
file_list = glob.glob(os.path.join(rootfolder, '*singlecut*.csv'), recursive=True)
file_list

['..\\..\\pfsp_experiments\\run_ccg_rpfs_wct_global\\separation_wct_hybrid_randomweights_10x2_warmstart-false_singlecut-false.csv',
 '..\\..\\pfsp_experiments\\run_ccg_rpfs_wct_global\\separation_wct_hybrid_randomweights_10x3_warmstart-false_singlecut-false.csv',
 '..\\..\\pfsp_experiments\\run_ccg_rpfs_wct_global\\separation_wct_hybrid_randomweights_10x4_warmstart-false_singlecut-false.csv',
 '..\\..\\pfsp_experiments\\run_ccg_rpfs_wct_global\\separation_wct_hybrid_randomweights_10x5_warmstart-false_singlecut-false.csv',
 '..\\..\\pfsp_experiments\\run_ccg_rpfs_wct_global\\separation_wct_hybrid_randomweights_15x5_warmstart-false_singlecut-true.csv',
 '..\\..\\pfsp_experiments\\run_ccg_rpfs_wct_global\\separation_wct_liao-you_randomweights_10x2_warmstart-false_singlecut-false.csv',
 '..\\..\\pfsp_experiments\\run_ccg_rpfs_wct_global\\separation_wct_liao-you_randomweights_10x3_warmstart-false_singlecut-false.csv',
 '..\\..\\pfsp_experiments\\run_ccg_rpfs_wct_global\\separation_wct_liao-

### Read all the CSV files 

In [98]:
# Alternative script to treat files with incorrect number of coluns or faulty lines
def alternative_csv_reader(filename, delimiter=',', header=0, names=None):
    with open(filename, 'r') as file:
        lines = file.readlines() 
        count = 1
        line_list = []
        num_columns = 23
        for line_num, line in enumerate(lines):  # Strips the newline character 
            #print("line{}: {}".format(count, line.strip())) 
            nc = len(line.split(','))
            if 'executionId,' in line:
                #num_columns = nc
                print('Detected {0} columns in CSV file.'.format(nc))
                if nc == num_columns - 1 and line_num == 0:
                    line_list.append(line + ',cut_count')
                # end if
            else:
                if 'none,' in line:
                    if nc == num_columns - 1:
                        line_list.append(line) #  + ',')
                    elif nc == num_columns:
                        line_list.append(line)
                    elif nc > num_columns:  # treat strange truncated lines
                        line = line[line.rfind('none,'):]
                        nc = len(line.split(','))
                        #if nc >= num_columns:
                        #    print('WARN: truncating line {0}, for having more columns than expected.'.format(count))
                        #    line_list.append(line)
                        #else:
                        print('WARN: Ignoring line {0}, since it has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
                    else:  # Ignore line
                        print('WARN: Ignoring line {0}: '.format(count), line)  
                else:  # Ignore line
                    print('WARN: Ignoring line {0}: '.format(count), line)
            count += 1
        # assert all lines have the same number of columns
        count = 1
        for line in line_list:
            nc = len(line.split(','))
            #if nc < num_columns:
            #    print('ERROR: Line {0} has {1} columns, instead of {2}: '.format(count, nc, num_columns), line)
            count += 1
        text_data = StringIO(''.join(line_list))
        #print('line_list: ', str(line_list))
        #print('text_data: ', text_data)
        df = pd.read_csv(text_data, delimiter=delimiter, header=header, names=names)
        return df

### Process all CSV files and append all data to a single dataframe (one per solution method: Wilson, Wagner) 

In [99]:
%%time

dfdict = dict()
for filepath in file_list:
    print('Processing file ', filepath)
    try:
        df_ = pd.read_csv(filepath, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','wct','permutation','time_spent','time_to_best_sol','mp_total_time','sp_total_time','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','wct_validation','cut_count'])
    except:  # try alternative method to read csv lines
        df_ = alternative_csv_reader(filepath, delimiter=',', header=0, names=['executionId','ub_name','instance_name','alpha','n','m','budget_Gamma','wct','permutation','time_spent','time_to_best_sol','mp_total_time','sp_total_time','iterations','num_visited_solutions','num_improvements','is_optimal','validated','gap','lb','cost','wct_validation','cut_count'])
    filename = filepath[filepath.rfind(os.path.sep)+1:]
    modelname = filename[len('separation_wct_'):filename.find('_randomweights')]
    print('Read results for model ' + modelname)
    if modelname in dfdict:
        dfdict[modelname] = pd.concat([dfdict[modelname], df_])
    else:
        dfdict[modelname] = df_.copy()

Processing file  ..\..\pfsp_experiments\run_ccg_rpfs_wct_global\separation_wct_hybrid_randomweights_10x2_warmstart-false_singlecut-false.csv
Read results for model hybrid
Processing file  ..\..\pfsp_experiments\run_ccg_rpfs_wct_global\separation_wct_hybrid_randomweights_10x3_warmstart-false_singlecut-false.csv
Read results for model hybrid
Processing file  ..\..\pfsp_experiments\run_ccg_rpfs_wct_global\separation_wct_hybrid_randomweights_10x4_warmstart-false_singlecut-false.csv
Read results for model hybrid
Processing file  ..\..\pfsp_experiments\run_ccg_rpfs_wct_global\separation_wct_hybrid_randomweights_10x5_warmstart-false_singlecut-false.csv
Read results for model hybrid
Processing file  ..\..\pfsp_experiments\run_ccg_rpfs_wct_global\separation_wct_hybrid_randomweights_15x5_warmstart-false_singlecut-true.csv
Read results for model hybrid
Processing file  ..\..\pfsp_experiments\run_ccg_rpfs_wct_global\separation_wct_liao-you_randomweights_10x2_warmstart-false_singlecut-false.csv
Det

In [100]:
dfdict

{'hybrid':    executionId          ub_name                        instance_name  alpha  \
 0         none   mip_separation     RB0101001_10_2_10_wct_inputs.txt     10   
 1         none   mip_separation     RB0101001_10_2_10_wct_inputs.txt     10   
 2         none   mip_separation     RB0101001_10_2_10_wct_inputs.txt     10   
 3         none   mip_separation     RB0101001_10_2_10_wct_inputs.txt     10   
 4         none   mip_separation     RB0101001_10_2_10_wct_inputs.txt     10   
 ..         ...              ...                                  ...    ...   
 15        none   mip_separation   RB0151001_15_5_R100_wct_inputs.txt      0   
 16        none   mip_separation   RB0151001_15_5_R100_wct_inputs.txt      0   
 17        none   mip_separation   RB0151001_15_5_R100_wct_inputs.txt      0   
 18        none   mip_separation   RB0151001_15_5_R100_wct_inputs.txt      0   
 19        none   mip_separation   RB0151001_15_5_R100_wct_inputs.txt      0   
 
      n  m  budget_Gamma    

In [101]:
dfdict['manne'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4317 entries, 0 to 215
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   executionId            4313 non-null   object
 1   ub_name                4317 non-null   object
 2   instance_name          4317 non-null   object
 3   alpha                  4317 non-null   object
 4   n                      4317 non-null   object
 5   m                      4317 non-null   object
 6   budget_Gamma           4317 non-null   object
 7   wct                    4317 non-null   object
 8   permutation            4317 non-null   object
 9   time_spent             4317 non-null   object
 10  time_to_best_sol       4317 non-null   object
 11  mp_total_time          4317 non-null   object
 12  sp_total_time          4317 non-null   object
 13  iterations             4317 non-null   object
 14  num_visited_solutions  4317 non-null   object
 15  num_improvements      

### Remove duplicated header rows from both dataframes 

In [102]:
def find_invalid_values(df):
    all_invalid_values = set()
    for col in df:
        if col not in ['executionId','ub_name','instance_name','budget_Gamma','permutation','is_optimal','validated']:
            # 'alpha','n','m','cmax','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','cmax_dp'
            a = pd.to_numeric(df[col], errors='coerce')
            idx = a.isna()
            invalid_values = df.loc[idx][col].unique()
            all_invalid_values.update(invalid_values)
        #elif col in ['is_optimal','validated']
    print('Invalid values:', all_invalid_values)
    return all_invalid_values

In [103]:
for key, df in dfdict.items():
    dfdict[key] = df[(df['executionId'] != 'executionId')]
    print(key, dfdict[key].dtypes)

hybrid executionId               object
ub_name                   object
instance_name             object
alpha                      int64
n                          int64
m                          int64
budget_Gamma             float64
wct                      float64
permutation               object
time_spent               float64
time_to_best_sol         float64
mp_total_time            float64
sp_total_time            float64
iterations                 int64
num_visited_solutions      int64
num_improvements           int64
is_optimal                object
validated                 object
gap                      float64
lb                       float64
cost                     float64
wct_validation           float64
cut_count                  int64
dtype: object
liao-you executionId              object
ub_name                  object
instance_name            object
alpha                    object
n                        object
m                        object
budget_Gamma       

### Convert column types from object 

In [104]:
def convert_column_types(df):
    for col in df:
        if col in ['alpha','n','m','wct','budget_Gamma','time_spent','time_to_best_sol','iterations','num_visited_solutions','num_improvements','gap','lb','cost','wct_validation', 'mp_total_time', 'sp_total_time', 'seq']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif col in ['is_optimal','validated']:
            df[col] = df[col].astype('bool')
    return df

In [105]:
%%time
for key, df in dfdict.items():
    dfdict[key] = convert_column_types(df)
    print(key, dfdict[key].dtypes)

hybrid executionId               object
ub_name                   object
instance_name             object
alpha                      int64
n                          int64
m                          int64
budget_Gamma             float64
wct                      float64
permutation               object
time_spent               float64
time_to_best_sol         float64
mp_total_time            float64
sp_total_time            float64
iterations                 int64
num_visited_solutions      int64
num_improvements           int64
is_optimal                  bool
validated                   bool
gap                      float64
lb                       float64
cost                     float64
wct_validation           float64
cut_count                  int64
dtype: object
liao-you executionId               object
ub_name                   object
instance_name             object
alpha                    float64
n                          int64
m                          int64
budget_Gamma 

### Trim existing string columns 

In [106]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [107]:
for key, df in dfdict.items():
    dfdict[key] = trim_all_columns(df)
    print(key)

hybrid
liao-you
manne
tba
ts2
ts3
wagner-wst2
wilson


### Include a column with the name of the underlying C&CG MILP Model

In [108]:
for key, df in dfdict.items():
    dfdict[key]['model'] = key

### Include a column with the name of the instance type (ying or tail)

In [109]:
for key, df in dfdict.items():
    dfdict[key]['instance_type'] = df['instance_name'].apply(lambda x: 'tail' if (x[0:5] == 'tail') else 'ying')

### Fix budget_Gamma paremeter value on Wilson model, `n=10`, `m=2`

In [110]:
for key, df in dfdict.items():
    if key == 'wilson':
        dfdict[key]['budget_Gamma'] = df.apply(lambda x: ((x['budget_Gamma']*x['m']*x['n'])/100.0) if (x['m'] == 2 and x['n'] == 10) else x['budget_Gamma'], axis=1)

### Include a column with the budget parameter value in %

In [111]:
for key, df in dfdict.items():
    dfdict[key]['Gamma%'] = 100.0 * df['budget_Gamma'] / (df['n'] * df['m'])

### Concatenate dataframes 

In [134]:
df = pd.concat(list(dfdict.values()))
df.head(4)

,executionId,ub_name,instance_name,alpha,n,m,budget_Gamma,wct,permutation,time_spent,...,is_optimal,validated,gap,lb,cost,wct_validation,cut_count,model,instance_type,Gamma%
0,none,mip_separation,RB0101001_10_2_10_wct_inputs.txt,10.0,10,2,2.0,64014.2,7 9 1 6 10 4 3 8 5 2,68.214676,...,True,True,1.136616e-16,64014.2,64014.2,64014.2,1,hybrid,ying,10.0
1,none,mip_separation,RB0101001_10_2_10_wct_inputs.txt,10.0,10,2,4.0,65550.0,7 9 1 6 10 4 3 8 5 2,23.795149,...,True,True,0.000000e+00,65550.0,65550.0,65550.0,1,hybrid,ying,20.0
2,none,mip_separation,RB0101001_10_2_10_wct_inputs.txt,10.0,10,2,6.0,66175.1,7 9 1 6 10 4 3 8 5 2,24.475667,...,True,True,0.000000e+00,66175.1,66175.1,66175.1,1,hybrid,ying,30.0
3,none,mip_separation,RB0101001_10_2_10_wct_inputs.txt,10.0,10,2,8.0,66617.5,7 9 1 6 10 4 3 8 5 2,21.865895,...,True,True,0.000000e+00,66617.5,66617.5,66617.5,1,hybrid,ying,40.0


### Fix instance names 

The original instance names, in the instance file zip, were assembled incorrectly.

The problem lies in the alpha percentage. We are now going to fix this issue.

In [135]:
df['seq'] = df['instance_name'].apply(lambda x: x[x.find('_')-2:x.find('_')])
df['n_str'] = df['n'].astype(str).str.zfill(3)
df['alpha_str'] = df['alpha'].astype(str)

### Round columns containing time (in seconds) 

In [136]:
df['time_spent'] = df['time_spent'].round(2)
df['time_to_best_sol'] = df['time_to_best_sol'].round(2)
df['mp_total_time'] = df['mp_total_time'].round(2)
df['sp_total_time'] = df['sp_total_time'].round(2)

### Drop duplicate rows, preserving the newest (last) ones

In [137]:
key_columns = ['model', 'n', 'm', 'alpha', 'seq', 'budget_Gamma', 'instance_type']
df.drop_duplicates(subset=key_columns, keep='last', inplace=True)

### Sort data according to model, instance_name, alpha, n, m and Gamma and set index

In [138]:
print('Sorting dataset...')
df = df.sort_values(key_columns)
display(df.dtypes)
df = df.set_index(key_columns, verify_integrity=True)
display(df.head(6))

Sorting dataset...


executionId               object
ub_name                   object
instance_name             object
alpha                    float64
n                          int64
m                          int64
budget_Gamma             float64
wct                      float64
permutation               object
time_spent               float64
time_to_best_sol         float64
mp_total_time            float64
sp_total_time            float64
iterations               float64
num_visited_solutions      int64
num_improvements           int64
is_optimal                  bool
validated                   bool
gap                      float64
lb                       float64
cost                     float64
wct_validation           float64
cut_count                 object
model                     object
instance_type             object
Gamma%                   float64
seq                       object
n_str                     object
alpha_str                 object
dtype: object

executionId         ub_name  \
model  n  m alpha seq budget_Gamma instance_type                               
hybrid 10 2 0.0   01  2.0          ying                 none  mip_separation   
                      4.0          ying                 none  mip_separation   
                      6.0          ying                 none  mip_separation   
                      8.0          ying                 none  mip_separation   
                      10.0         ying                 none  mip_separation   
                      12.0         ying                 none  mip_separation   

                                                                       instance_name  \
model  n  m alpha seq budget_Gamma instance_type                                       
hybrid 10 2 0.0   01  2.0          ying           RB0101001_10_2_R400_wct_inputs.txt   
                      4.0          ying           RB0101001_10_2_R400_wct_inputs.txt   
                      6.0          ying           RB0101001_10_2_R400_wct_inputs.txt   
                      8.0          ying           RB0101001_10_2_R400_wct_inputs.txt   
                      10.0         ying           RB0101001_10_2_R400_wct_inputs.txt   
                      12.0         ying           RB0101001_10_2_R400_wct_inputs.txt   

                                                        wct  \
model  n  m alpha seq budget_Gamma instance_type              
hybrid 10 2 0.0   01  2.0          ying           116043.57   
                      4.0          ying           153238.39   
                      6.0          ying           175669.94   
                      8.0          ying           187399.41   
                      10.0         ying           194179.33   
                      12.0         ying           199098.52   

                                                           permutation  \
model  n  m alpha seq budget_Gamma instance_type                         
hybrid 10 2 0.0   01  2.0          ying           1 3 7 6 8 10 9 4 2 5   
                      4.0          ying           10 7 8 1 3 6 9 4 2 5   
                      6.0          ying           10 7 8 6 1 3 9 4 2 5   
                      8.0          ying           10 7 6 8 1 9 3 4 2 5   
                      10.0         ying           7 6 10 8 1 9 3 4 2 5   
                      12.0         ying           7 6 10 8 1 9 3 4 2 5   

                                                  time_spent  \
model  n  m alpha seq budget_Gamma instance_type               
hybrid 10 2 0.0   01  2.0          ying               405.70   
                      4.0          ying               581.04   
                      6.0          ying               814.50   
                      8.0          ying              1177.09   
                      10.0         ying               758.89   
                      12.0         ying               639.23   

                                                  time_to_best_sol  \
model  n  m alpha seq budget_Gamma instance_type                     
hybrid 10 2 0.0   01  2.0          ying                     405.70   
                      4.0          ying                     581.04   
                      6.0          ying                     814.50   
                      8.0          ying                    1177.09   
                      10.0         ying                     758.89   
                      12.0         ying                     639.23   

                                                  mp_total_time  \
model  n  m alpha seq budget_Gamma instance_type                  
hybrid 10 2 0.0   01  2.0          ying                  405.40   
                      4.0          ying                  579.71   
                      6.0          ying                  812.23   
                      8.0          ying                 1174.18   
                      10.0         ying                  756.83   
                      12.0         ying                  637.5

### Find missing results, for a given value of alpha, n and m

For a given group of alpha, n, m and budget_Gamma, there should be 10 results.

First we will build a dataframe with the instances list and all required budget values.

In [139]:
data = []
rootfolder = os.getcwd()
file_list = Path(os.path.join(rootfolder, 'instances', 'robust')).rglob('*.txt')
file_set = set()
for path in file_list:
    instance_path = path.name
    #print('instance_path: ' + instance_path)    
    if '.txt' not in instance_path:
        continue
    if 'tail' in instance_path:
        if instance_path[:instance_path.find('_')] not in ['tail001', 'tail002', 'tail003', 'tail004', 'tail005', 'tail006', 'tail007', 'tail008', 'tail009', 'tail010']:
            #print(instance_path[:instance_path.find('_')])
            continue
    instance_name = instance_path[instance_path.rfind(os.path.sep)+1:]
    file_set.add(instance_name)
#print(file_set, file_set)
for instance_name in file_set:
    #print('instance_name: ' + instance_name)
    seq = instance_name[instance_name.find('_')-2:instance_name.find('_')]
    info = instance_name[instance_name.find('_')+1:]
    n = info[:info.find('_')]
    info = info[info.find('_')+1:]
    m = info[:info.find('_')]
    info = info[info.find('_')+1:]
    alpha = info[:info.find('_')]
    instance_type = 'ying'
    if 'tail' in instance_name:
        instance_type = 'tail'
    else:
        if int(n) > 20:
            continue
    for gamma in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
        budget_gamma = int(gamma) * (int(m) * int(n)) / 100.0
        for model in list(dfdict.keys()):
            data.append([model, seq, alpha, int(n), int(m), budget_gamma, instance_type])
df_instances = pd.DataFrame(data, columns=['model', 'seq', 'alpha', 'n', 'm', 'budget_Gamma', 'instance_type'])
for col in df_instances:
    if col in ['alpha','n','m','budget_Gamma']:
        df_instances[col] = pd.to_numeric(df_instances[col], errors='coerce')
display(df_instances.dtypes)
df_instances = df_instances.set_index(['model', 'n', 'm', 'alpha', 'seq', 'budget_Gamma', 'instance_type'], verify_integrity=True)
display(df_instances)

model             object
seq               object
alpha              int64
n                  int64
m                  int64
budget_Gamma     float64
instance_type     object
dtype: object

,,,,,,
model,n,m,alpha,seq,budget_Gamma,instance_type
hybrid,10,5,20,09,5.0,ying
liao-you,10,5,20,09,5.0,ying
manne,10,5,20,09,5.0,ying
tba,10,5,20,09,5.0,ying
ts2,10,5,20,09,5.0,ying
...,...,...,...,...,...,...
tba,10,4,20,03,40.0,ying
ts2,10,4,20,03,40.0,ying
ts3,10,4,20,03,40.0,ying


In [140]:
df_instances.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 36000 entries, ('hybrid', 10, 5, 20, '09', 5.0, 'ying') to ('wilson', 10, 4, 20, '03', 40.0, 'ying')
Empty DataFrame

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 18419 entries, ('hybrid', 10, 2, 0.0, '01', 2.0, 'ying') to ('wilson', 15, 5, 50.0, '01', 75.0, 'ying')
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   executionId            18391 non-null  object 
 1   ub_name                18419 non-null  object 
 2   instance_name          18419 non-null  object 
 3   wct                    18414 non-null  float64
 4   permutation            18419 non-null  object 
 5   time_spent             18391 non-null  float64
 6   time_to_best_sol       18419 non-null  float64
 7   mp_total_time          18419 non-null  float64
 8   sp_total_time          18419 non-null  float64
 9   iterations             18419 non-null  float64
 10  num_visited_solutions  18419 non-null  int64  
 11  num_improvements       18419 non-null  int64  
 12  is_optimal             18419 non-null  bool   
 13  validated              18419

Now, lets join the instances dataframe with the results one (left join).

In [142]:
df_joined = df_instances.join(df, how='left')
df_joined

,,,,,,,executionId,ub_name,instance_name,wct,permutation,time_spent,time_to_best_sol,mp_total_time,sp_total_time,iterations,...,is_optimal,validated,gap,lb,cost,wct_validation,cut_count,Gamma%,n_str,alpha_str
model,n,m,alpha,seq,budget_Gamma,instance_type,,,,,,,,,,,,,,,,,,,,,
hybrid,10,5,20,09,5.0,ying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
liao-you,10,5,20,09,5.0,ying,none,mip_separation,RB0102009_10_5_20_wct_inputs.txt,145546.8,7 1 5 10 8 3 2 4 9 6,813.47,813.47,677.81,544.27,8.0,...,True,True,1.999620e-16,145546.8,145546.8,145546.8,NaN,10.0,010,20.0
manne,10,5,20,09,5.0,ying,none,mip_separation,RB0102009_10_5_20_wct_inputs.txt,145546.8,7 1 5 10 8 3 2 4 9 6,561.48,561.48,86.30,400.74,6.0,...,True,True,3.999240e-16,145546.8,145546.8,145546.8,NaN,10.0,010,20.0
tba,10,5,20,09,5.0,ying,none,mip_separation,RB0102009_10_5_20_wct_inputs.txt,145546.8,7 1 5 10 8 3 2 4 9 6,2561.06,2561.06,4499.54,496.75,6.0,...,True,True,-1.999620e-16,145546.8,145546.8,145546.8,NaN,10.0,010,20.0
ts2,10,5,20,09,5.0,ying,none,mip_separation,RB0102009_10_5_20_wct_inputs.txt,145546.8,7 1 5 10 8 3 2 4 9 6,1800.47,1800.47,2673.02,853.84,8.0,...,True,True,1.999620e-16,145546.8,145546.8,145546.8,NaN,10.0,010,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tba,10,4,20,03,40.0,ying,none,mip_separation,RB0101003_010_004_20_wct_inputs.txt,156207.6,4 2 3 9 6 8 5 1 7 10,441.22,441.22,430.44,7.17,2.0,...,True,True,1.863151e-16,156207.6,156207.6,156207.6,NaN,100.0,010,20.0
ts2,10,4,20,03,40.0,ying,none,mip_separation,RB0101003_010_004_20_wct_inputs.txt,156207.6,4 2 3 9 6 8 5 1 7 10,221.05,221.05,239.21,12.74,2.0,...,True,True,3.726301e-16,156207.6,156207.6,156207.6,NaN,100.0,010,20.0
ts3,10,4,20,03,40.0,ying,none,mip_separation,RB0101003_010_004_20_wct_inputs.txt,156207.6,4 2 3 9 6 8 5 1 7 10,93.74,93.74,84.88,5.81,2.0,...,True,True,0.000000e+00,156207.6,156207.6,156207.6,NaN,100.0,010,20.0


In [143]:
df_joined.reset_index()[['model', 'n', 'm', 'alpha', 'seq', 'budget_Gamma', 'instance_type']]

,model,n,m,alpha,seq,budget_Gamma,instance_type
0,hybrid,10,5,20,09,5.0,ying
1,liao-you,10,5,20,09,5.0,ying
2,manne,10,5,20,09,5.0,ying
3,tba,10,5,20,09,5.0,ying
4,ts2,10,5,20,09,5.0,ying
...,...,...,...,...,...,...,...
35995,tba,10,4,20,03,40.0,ying
35996,ts2,10,4,20,03,40.0,ying
35997,ts3,10,4,20,03,40.0,ying
35998,wagner-wst2,10,4,20,03,40.0,ying


Now we will export to CSV a list with all rows with NaN values (missing experimental results).

In [144]:
missing_df = df_joined[df_joined[['wct']].isnull().any(axis=1)].reset_index()[['model', 'n', 'm', 'alpha', 'seq', 'budget_Gamma', 'instance_type']]
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder)
print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, 'RPFS_TWCT_missing_results.csv')
missing_df.to_csv(fname, sep=';')
print('Saved: ' + fname)

Saving file on folder: C:\Users\Public\projetos\RPFS_Budget_TWCT\results\consolidated
Saved: C:\Users\Public\projetos\RPFS_Budget_TWCT\results\consolidated\RPFS_TWCT_missing_results.csv


In [149]:
df_grouped = df.groupby(['alpha', 'n', 'm', 'Gamma%']).agg({'executionId' : ['count']}).reset_index()
df_grouped.columns = [ ' '.join(str(i) for i in col) for col in df_grouped.columns]
#df_grouped.reset_index(inplace=True)
df_grouped

,alpha,n,m,Gamma%,executionId count
0,0.0,10,2,2.0,10
1,0.0,10,2,4.0,10
2,0.0,10,2,6.0,10
3,0.0,10,2,8.0,10
4,0.0,10,2,10.0,80
...,...,...,...,...,...
323,50.0,15,5,60.0,7
324,50.0,15,5,70.0,7
325,50.0,15,5,80.0,7
326,50.0,15,5,90.0,7


In [154]:
perc_list = [x * 10 for x in range(1, 11)]
table = pd.pivot_table(df[(df['Gamma%'].isin(perc_list))], values='executionId', index=['alpha', 'n', 'm', 'model'], columns=['Gamma%'], aggfunc='count', fill_value=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(table)

Gamma%                  10.0   20.0   30.0   40.0   50.0   60.0   70.0   \
alpha n  m model                                                          
0.0   10 2 hybrid          10     10     10     10     10     10     10   
           liao-you        10     10     10     10     10     10     10   
           manne           10     10     10     10     10     10     10   
           tba             10     10     10     10     10     10     10   
           ts2             10     10     10     10     10     10     10   
           ts3             10     10     10     10     10     10     10   
           wagner-wst2     10     10     10     10     10     10     10   
           wilson          10     10      0      0      0      0      0   
         3 hybrid           6      6      6      6      6      6      6   
           liao-you        10     10     10     10     10     10     10   
           manne           10     10     10     10     10     10     10   
           tba             10     10     10     10     10     10     10   
           ts2             10     10     10     10     10     10     10   
           ts3             10     10     10     10     10     10     10   
           wagner-wst2     10     10     10     10     10     10     10   
           wilson          10     10     10     10     10     10     10   
         4 hybrid           3      3      3      3      3      3      3   
           liao-you        10     10     10     10     10     10     10   
           manne           10     10     10     10     10     10     10   
           tba             10     10     10     10     10     10     10   
           ts2             10     10     10     10     10     10     10   
           ts3             10     10     10     10     10     10     10   
           wagner-wst2     10     10     10     10     10     10     10   
           wilson          10     10     10     10     10     10     10   
         5 hybrid           3      3      3      3      3      3      3   
           liao-you        10     10     10     10     10     10     10   
           manne           10     10     10     10     10     10     10   
           tba             10     10     10     10     10     10     10   
           ts2             10     10     10     10     10     10     10   
           ts3             10     10     10     10     10     10     10   
           wagner-wst2     10     10     10     10     10     10     10   
           wilson          10     10     10     10     10     10     10   
      15 5 hybrid           1      1      1      1      1      1      1   
           liao-you         4      4      4      4      4      4      4   
           manne            4      4      4      4      4      4      4   
           ts2              1      1      1      1      1      1      1   
           wilson           1      1      1      1      1      1      1   
10.0  10 2 hybrid          10     10     10     10     10     10     10   
           liao-you         9     10     10     10     10     10     10   
           manne            9     10     10     10     10     10     10   
           tba              9     10     10     10     10     10     10   
           ts2              9     10     10     10     10     10     10   
           ts3              9     10     10     10     10     10     10   
           wagner-wst2      9     10     10     10     10     10     10   
           wilson          10     10      0      0      0      0      0   
         3 hybrid           6      6      6      6      6      6      6   
           liao-you         9     10     10     10     10     10     10   
           manne            9     10     10     10     10     10     10   
           tba              9     10     10     10     10     10     10   
           ts2              9     10     10     10     10     10     10   
           ts3             10     10     10     10     10     10     10   
           wagner-wst2   

### Export the dataset to CSV file 

In [155]:
%%time

outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
print('Saving file on folder: ' + outputfolder)
fname = os.path.join(outputfolder, 'RPFS_TWCT_all_results.csv')
df.to_csv(fname, sep=';')
print('Saved: ' + fname)
fname = os.path.join(outputfolder, 'RPFS_TWCT_all_results.pkl.gz')
df.to_pickle(fname)
print('Saved: ' + fname)

Saving file on folder: C:\Users\Public\projetos\RPFS_Budget_TWCT\results\consolidated
Saved: C:\Users\Public\projetos\RPFS_Budget_TWCT\results\consolidated\RPFS_TWCT_all_results.csv
Saved: C:\Users\Public\projetos\RPFS_Budget_TWCT\results\consolidated\RPFS_TWCT_all_results.pkl.gz
Wall time: 1.26 s
